In [1]:
import torch
import torch.nn as nn

#Network implementation
class Byter(nn.Module):
    def __init__(self):
        super(Byter, self).__init__()
        
    
    def forward(x):
        pass

ValueError: libcublas.so.*[0-9] not found in the system path ['/home/nullptr/anaconda3/envs/ml/lib/python312.zip', '/home/nullptr/anaconda3/envs/ml/lib/python3.12', '/home/nullptr/anaconda3/envs/ml/lib/python3.12/lib-dynload', '', '/home/nullptr/anaconda3/envs/ml/lib/python3.12/site-packages', '/home/nullptr/anaconda3/envs/ml/lib/python3.12/site-packages/setuptools/_vendor']

In [ ]:
b = Byter()

optimizer = torch.optim.Adam(Byter.parameters(), lr = 0.01)

for i in range(1000):
    x, y = sample(training_set)
    b(x)
    l = loss(x, y)
    l.backward()
    if(i%10):
        optimizer.step()